<h1 style="color: #8b5e3c;">Bucks Model Development</h1>
Next, we will be working on developing a predictive model that helps assign each user account for one of tucket plans that the Miwaukee Bucks is interested in. We revisit the plans as follows:

- **Value Plan:** focuses on affordable tickets for weekday games
- **Marquee Opponent Plan:** featuring games against high-profile opponents
- **Weekend Plan:** highlighting weekend games for fans looking for weekend entertainment
- **Promotional Giveaway Inclusive Plan:** centered around games with promotional giveaways

<h2 style="color: #8b5e3c;"> Using K-Means Clustering </h2>
We start off by using K-Means Clustering on your dataset. This will help us find the clusters we're interested in, and then look into the clusters to see what kind of information we can derive from it.